In [1]:
# import libraries
import pandas as pd
from sklearn.impute import SimpleImputer

### Import the data

In [2]:
tnm1 = pd.read_csv("breast_cancer_data_tnm.csv")
tnm2 = pd.read_csv("breast_cancer_data_tnm_2.csv")

df1 = pd.read_excel("breast_cancer_data.xlsx")
df2 = pd.read_excel("breast_cancer_data_2.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'breast_cancer_data_tnm.csv'

In [ ]:
# concatenate the two datasets
data = pd.concat([df1, df2])
data

In [ ]:
# concatenate the two datasets
data_tnm = pd.concat([tnm1, tnm2])
data_tnm

### NULL values

In [ ]:
print(data.isnull().sum())

In [ ]:
print(data_tnm.isnull().sum())

In [ ]:
# drop the first col (index) and side 
data.pop('Unnamed: 0')
data.pop('side')
data.head()

In [ ]:
data.info()

### Adding new columns

In [ ]:
data['Alive'] = data.apply(lambda row: 1 if(pd.isna(row.death_date)) else 0 , axis=1)
data.head(20)

In [ ]:
# For menarche_age we can use the most frequent one.
data = data.fillna(data['menarche_age'].value_counts().index[0])
data

In [ ]:
# Split dataFrame in two: categorical and numerical:
cat_mask = (data.dtypes == object)
cat_cols = data.columns[cat_mask].tolist()

df_cat = data[cat_cols]
df_num = data.drop(cat_cols, axis=1)

df_cat.info()
df_num.info()

In [ ]:
# For categorical variables
imp_cat = SimpleImputer(strategy='most_frequent')
columns = df_cat.columns
index = df_cat.index
df_cat = pd.DataFrame(imp_cat.fit_transform(df_cat), columns = columns, index = index)

print(df_cat.isnull().sum())

In [ ]:
df_cat.head(20)